In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import xgboost as xgb
#https://habrahabr.ru/company/ods/blog/327242/

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
%matplotlib inline
import seaborn as sb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.offsets import *

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import __version__
from plotly import graph_objs as go

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

C:\Users\TEMP\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Читаем данные
df=pd.read_pickle('MG_Sales.pickle',compression='gzip')
celebrate=pd.read_pickle('celebrate.pickle')

In [78]:
#готовим выборку обучения
begin_period=dt.datetime(2015,1,1)
end_period=dt.datetime(2017,3,31)


start_forecast=end_period+dt.timedelta(days=1)
end_forecast=start_forecast+dt.timedelta(days=30)

query="Дата>=20150101 and ТоварЦеноваяГруппа=='2100-3200'"
time_series=pd.DataFrame(data=df.query(query).groupby('Дата')['Количество'].sum())
time_series.index.name='ds'
time_series.columns=['y']
time_series.y=time_series.y

#Пустой период в днях обучающей выборки
delta=end_period-begin_period
dummy_train = pd.DataFrame(index=np.array([begin_period + dt.timedelta(days=x) for x in range(0, delta.days)]).astype('datetime64[D]'))
dummy_train.index.name='ds'

#сливаем обучающую выборку и пустой период чтобы избежать пропусков дат, пропуски заполняем нулями
time_series=dummy_train.merge(time_series,left_index=True, right_index=True,how='outer').fillna(0)  

In [79]:
month_lags=1

def setPercentil(row,percentiles):
    if row<percentiles['llim']: return -2
    if row>percentiles['ulim']: return 2
    if row<percentiles['med'][0]: return -1
    if row>percentiles['med'][1]: return 1
    return 0

def getPercentil(ts):
    #Фиксируем аномально низкие и высокие продажи
    ul=5#Персентиль высоких продаж 2
    ll=5#7#Персентиль низких продаж 10
    md=25#ширина медианы
    percentiles={}
    
    percentiles['ulim']=np.percentile(ts['y'], 100.-ul)
    percentiles['llim']=np.percentile(ts['y'], ll)
    percentiles['med']=np.percentile(ts['y'], [50-md,50+md])
    return percentiles
    

#временные характеристики
def setNewValues(_ts,cel):
    ts=_ts.copy()
    y=None
    week_d=None
    last_tempo=None
    mean_day_year=None
    percentiles={}
    
    ts['День недели'] = ts.index.weekday
    ts['Выходной'] = ((ts.index.weekday.isin([5,6]))|(ts.index.isin(cel['Праздник'])))*1#ts.index.weekday.isin([4])*1
    ts['Неделя'] = ts.index.week
    ts['Год'] = ts.index.year
    ts['Месяц'] = ts.index.month
    ts['День месяца'] = ts.index.day
    #ts['День года'] = ts.index.dayofyear
    #ts['Праздник']=0#Обычный день
    #ts.loc[ts.index.isin(cel['Праздник']),'Праздник']=1#праздник
    
    if 'y' in ts.columns:#ТОЛЬКО для обучающей выборки
        #Фомируем характеристики модели 
        percentiles=getPercentil(ts)#получаем распределение персентилей
        
        
        ts['Среднее по дню года']=0
        #Среднее по дню года, нарастающим
        for year in ts['Год'].unique():    
            mean_day_year=ts.loc[ts['Год']<=year,['Месяц','День месяца','y']].reset_index()#.drop('ds',axis=1)
    
            mean2829=np.mean(mean_day_year.loc[(mean_day_year['Месяц']==2)&(mean_day_year['День месяца'].isin([28,29])),'y'])
            mean2801=np.mean(mean_day_year.loc[(mean_day_year['Месяц']==2)&(mean_day_year['День месяца']==28)|(mean_day_year['Месяц']==3)&(mean_day_year['День месяца']==1),'y'])
    
            mean_day_year.loc[(mean_day_year['Месяц']==2)&(mean_day_year['День месяца']==28),'y']=mean2829
            mean_day_year.loc[(mean_day_year['Месяц']==2)&(mean_day_year['День месяца']==29),'y']=mean2801   

            mean_day_year=dict(mean_day_year.groupby(['Месяц','День месяца'])['y'].mean())
            ts['Среднее по дню года']=ts.apply(lambda row: mean_day_year[row['Месяц'],row['День месяца']] if row['Год']==year else row['Среднее по дню года'] , axis=1) 
        ts['Квантили']=ts.apply(lambda row: setPercentil(row['Среднее по дню года'],percentiles), axis=1)
        
                    
        #порядок дней в сезонности недельной продажи за исключением аномалий
        week_d=pd.DataFrame(data=ts[ts['Квантили']==0].groupby('День недели')['y'].sum().sort_values())
        week_d.insert(0,'Недельная сезонность',list(range(week_d.shape[0])))
        for i in list(set(range(7))-set(week_d.index.values)):
            week_d.loc[i,'Недельная сезонность']=-1
        ts=weekseason(ts,week_d)
    
        #Среднее и тренды                        
        mean_dict=dict(ts.groupby(['Год'])['y'].mean())
        ts['Среднее за год']=ts.apply(lambda row: mean_dict[row['Год']] , axis=1)        
                        
        #РАССЧИТЫВАЕМ коэффициент роста
        mean_month=ts.groupby(['Год','Месяц'])['y'].mean()
        mean_month_shift=mean_month.shift(1).fillna(0)
        #(mean_month/mean_month_shift-1)*mean_month.shift(11).fillna(0)
        inc_temp=(mean_month/mean_month_shift-1)#*mean_month.shift(11).fillna(0)        
        last_tempo=inc_temp.tail(1).values[0]
        
        #inc_temp_dict=dict(inc_temp)
        #ts['Месячный тренд']=ts.apply(lambda row: inc_temp_dict[(row['Год'],row['Месяц'])] , axis=1).replace(np.inf, 0)
        #print(ts['Месячный тренд'])
        
    
        #Вычленяем целевую переменную
        y=ts.y
        ts.drop(['y'], axis=1, inplace=True)
        
        #заглушки        
        ts.drop(['День недели'], axis=1, inplace=True)                
    
    return ts,y,week_d,mean_day_year,percentiles,last_tempo


def weekseason(ts,week_d):
    ts['Недельная сезонность']=ts['День недели'].map(lambda cell: week_d.loc[cell,'Недельная сезонность'])
    return ts

#Сдвигаем период на год вперед
def fillTimeSeriesForecast(_tsf,ts,week_d,mean_day_year,percentiles,last_tempo):
    tsf=weekseason(_tsf.copy(),week_d)
    tsf.drop(['День недели'], axis=1, inplace=True)
        
    tsf['Среднее по дню года']=tsf.apply(lambda row: mean_day_year[row['Месяц'],row['День месяца']], axis=1)
    tsf['Квантили']=tsf.apply(lambda row: setPercentil(row['Среднее по дню года'],percentiles), axis=1)
        
    #Мы знаем средние данные за год, учитываем как тренд
    last_vals=ts.tail(1)
    for col in ['Среднее за год']:
        tsf[col]=last_vals[col].values[0]
        
    #print('Темп прироста: ',last_tempo)
  #  tsf['Среднее по дню года']+=tsf['Среднее по дню года']*last_tempo
    #tsf['Месячный тренд']=last_tempo
    #tsf['Среднее за год']+=tsf['Среднее за год']*last_tempo
    
    
    return tsf[ts.columns]#Колонки в правильном порядке
    

def createTimeSeriesForecast(begin, end,cel):
    date_list = pd.date_range(begin, end).tolist()
    time_series_forecast=pd.DataFrame(index=date_list)
    time_series_forecast.index.name='ds'    
    
    time_series_forecast=setNewValues(time_series_forecast,cel)
    
    return time_series_forecast    

In [80]:
lastDay=end_period

time_series_train, y_train,week_d,mean_day_year,percentiles,last_tempo=setNewValues(time_series.loc[:lastDay],celebrate)
    
tss = TimeSeriesSplit(n_splits=10)
tss_cv=list(tss.split(time_series_train,y_train))

# задаём параметры
params = {
        'objective': 'reg:linear',
        'booster':'gblinear',                
        'eta': 0.23,#коэффициент обучения
        'alpha': 0.1,
        'lambda_bias': 0,
        'eval_metric': 'rmse'
    }

#Нормализуйте обучающую выборку с помощью класса StandardScaler
scaler = StandardScaler(with_mean=True,with_std=True)
dtrain = xgb.DMatrix(scaler.fit_transform(time_series_train), label=y_train)#

# прогоняем на кросс-валидации с метрикой rmse
cv = xgb.cv(params, dtrain, metrics = ('rmse'), early_stopping_rounds=50,verbose_eval=False, show_stdv=False, num_boost_round=1500,folds=tss_cv)#,nfold=10

mod_n=cv['test-rmse-mean'].argmin()

deviation = cv.loc[mod_n]["test-rmse-mean"]
best_iteration=mod_n

#Модель предсказания
time_series_forecast,_,_2,_3,_4,_5 =createTimeSeriesForecast(start_forecast, end_forecast,celebrate)
time_series_forecast=fillTimeSeriesForecast(time_series_forecast,time_series_train,week_d,mean_day_year,percentiles,last_tempo)

In [81]:
bst = xgb.train(params, dtrain, best_iteration,verbose_eval=False)
prediction_test = pd.DataFrame(data=bst.predict(xgb.DMatrix(scaler.transform(time_series_forecast))),index=time_series_forecast.index)#
prediction_test[0]=prediction_test[0].map(lambda val: 0 if val<0.01 else round(val,3))

print('Средняя ошибка на кросс-валидации: ',deviation,', номер модели: ',mod_n)
print('Темп прироста: ',last_tempo)

print('RSME = ',np.sqrt(mean_squared_error(y_test.y,prediction_test[0])))

Средняя ошибка на кросс-валидации:  10.0906821 , номер модели:  86
Темп прироста:  0.0646902877489
RSME =  47.4981996803


In [82]:
y_test=time_series[start_forecast:end_forecast]


init_notebook_mode(connected = True)
trace1 = go.Scatter(
            x = prediction_test.index,
            y = prediction_test[0],
            mode = 'lines',            
            name = 'Прогноз 2017',
            line=dict(
                shape='spline',
                dash = 'dash',
                width = 3
            )
        )       

ty=(time_series[dt.datetime(2016,1,1):dt.datetime(2016,6,1)].shift(1,DateOffset(years=1)))
trace0 = go.Scatter(
            x = ty.index,
            y = ty.y,
            mode = 'lines',            
            name = '2016 год',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        )       

ty=(time_series[dt.datetime(2015,1,1):dt.datetime(2015,6,1)].shift(2,DateOffset(years=1)))
trace5 = go.Scatter(
            x = ty.index,
            y = ty.y,
            mode = 'lines',            
            name = '2015 год',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        )   

y_test=time_series.loc[time_series_forecast.index]
trace2 = go.Scatter(
            x = y_test.index,
            y = y_test.y,
            mode = 'lines',            
            name = 'Продажи 2017',
            line=dict(
                shape='spline'
            )
        ) 
trace4 = go.Scatter(
            x = y_test.index,
            y = y_test.y-prediction_test[0],
            mode = 'markers',
            name = 'Разница прогноза и данных',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        ) 

layout = go.Layout(
    title=query
)
    
fig =go.Figure(data = [trace1,trace2,trace5,trace0,trace4],layout=layout)#,trace0
iplot(fig, show_link=False)



In [83]:
time_series_train[dt.datetime(2016,3,1):dt.datetime(2016,3,10)]

,Выходной,Неделя,Год,Месяц,День месяца,Среднее по дню года,Квантили,Недельная сезонность,Среднее за год
ds,,,,,,,,,
2016-03-01,0,9,2016,3,1,49.0,0,2,57.688525
2016-03-02,0,9,2016,3,2,77.0,1,6,57.688525
2016-03-03,0,9,2016,3,3,81.5,1,4,57.688525
2016-03-04,0,9,2016,3,4,101.0,1,5,57.688525
2016-03-05,1,9,2016,3,5,128.5,2,3,57.688525
2016-03-06,1,9,2016,3,6,136.0,2,1,57.688525
2016-03-07,1,10,2016,3,7,148.0,2,0,57.688525
2016-03-08,1,10,2016,3,8,100.5,1,2,57.688525
2016-03-09,0,10,2016,3,9,40.0,0,6,57.688525


In [85]:
time_series_forecast[dt.datetime(2017,4,1):dt.datetime(2017,4,10)]

,Выходной,Неделя,Год,Месяц,День месяца,Среднее по дню года,Квантили,Недельная сезонность,Среднее за год
ds,,,,,,,,,
2017-04-01,1,13,2017,4,1,47.0,0,3,56.555556
2017-04-02,1,13,2017,4,2,37.0,0,1,56.555556
2017-04-03,0,14,2017,4,3,30.0,-1,0,56.555556
2017-04-04,0,14,2017,4,4,51.5,0,2,56.555556
2017-04-05,0,14,2017,4,5,24.5,-1,6,56.555556
2017-04-06,0,14,2017,4,6,36.5,0,4,56.555556
2017-04-07,0,14,2017,4,7,37.0,0,5,56.555556
2017-04-08,1,14,2017,4,8,53.0,0,3,56.555556
2017-04-09,1,14,2017,4,9,36.0,0,1,56.555556
